In [ ]:
import numpy as np
import pandas as pd
import requests 
import math
!pip install xlsxwriter
import xlsxwriter
from google.colab import files
import io


     |████████████████████████████████| 149 kB 4.3 MB/s 


In [ ]:
uploaded = files.upload()
#IEX_API_TOKEN
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

#IMPORT STOCK LIST
stocks = pd.read_csv(io.BytesIO(uploaded['sp_500_stocks.csv']))

Saving sp_500_stocks.csv to sp_500_stocks.csv


In [ ]:
symbol='aapl'
api_url=f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data=requests.get(api_url).json()
print(data)

{'avgTotalVolume': 94528748, 'calculationPrice': 'close', 'change': 0.32, 'changePercent': 0.0018, 'close': 174.55, 'closeSource': 'iaffloci', 'closeTime': 1681587488969, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 180.183, 'delayedPriceTime': 1707920829625, 'extendedChange': 0.08, 'extendedChangePercent': 0.00047, 'extendedPrice': 173.69, 'extendedPriceTime': 1685175391692, 'high': 181.9, 'highSource': '1eyimuctd apedine5rl  e', 'highTime': 1651413750469, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 178.22, 'iexCloseTime': 1726150096178, 'iexLastUpdated': 1663774049170, 'iexMarketPercent': 0.019785044227395857, 'iexOpen': 177.93, 'iexOpenTime': 1660232484807, 'iexRealtimePrice': 178.22, 'iexRealtimeSize': 207, 'iexVolume': 1502742, 'lastTradeTime': 1679056084189, 'latestPrice': 173.64, 'latestSource': 'Close', 'latestTime': 'April 7, 2022', 'latestUpdate': 1653320110587, 'latestVolume': 79022387, 'low': 173.17, 'lowSource': 'c5u1

**`Parsing the Api call`**
`

Consider the peRatio

In [ ]:
def chunks(lst, n):
  #yeilds succesive n sized chunks from lists
  for i in range (0,len(lst),n):
    yield lst[i: i+n]
symbol_groups= list(chunks(stocks['Ticker'],100))
symbol_strings=[]
for i in range(0,len(symbol_groups)):
  symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker','Companys Name', 'Price-to-Earnings Ratio', 'Number Of Shares to Buy']

In [ ]:
final_dataframe=pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
  batch_api_call_url= f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(','):
    if symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW' or symbol=='DISCA':
           continue
    else:
      final_dataframe=final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['peRatio'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [ ]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index( inplace=True, drop= True)

In [ ]:
def portfolio_input():
  global portfolio_size
  portfolio_size=input("Enter the size of the portfolio: ")
  try:
    float(portfolio_size)
  except ValueError:
    print("That's not a number! Enter an integer value eg:- 100000,100,555 :")
    portfolio_size=input("Enter the size of the portfolio: ")

portfolio_input()

Enter the size of the portfolio: 1000000000


**Check out all the ratios for 1 stock**

In [ ]:
symbol = 'AAL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit
print(ev_to_gross_profit)

1.2001500173767827


**RV down there stands for ROBUST VALUE**

In [ ]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [ ]:

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
      if symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW' or symbol=='DISCA':
           continue
      else:
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [ ]:
len(rv_dataframe[rv_dataframe.isnull().any(axis=1)].index)


19

In [ ]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
  rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

In [ ]:
len(rv_dataframe[rv_dataframe.isnull().any(axis=1)].index)

0

In [ ]:
import scipy
from scipy.stats import percentileofscore
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = scipy.stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])
   
rv_dataframe

0      0.744511
1       0.03992
2      0.479042
3      0.618762
4      0.602794
         ...   
496    0.491018
497    0.942116
498    0.600798
499    0.095808
500    0.842315
Name: PE Percentile, Length: 501, dtype: object
0      0.786427
1       0.05988
2        0.5998
3      0.974052
4      0.932136
         ...   
496    0.055888
497    0.325349
498    0.783433
499    0.199601
500     0.94012
Name: PB Percentile, Length: 501, dtype: object
0      0.808383
1       0.01996
2      0.157685
3      0.846307
4      0.748503
         ...   
496    0.729541
497    0.531936
498    0.602794
499    0.438124
500     0.92016
Name: PS Percentile, Length: 501, dtype: object
0      0.790419
1      0.053892
2      0.403194
3      0.774451
4       0.45509
         ...   
496    0.700599
497    0.714571
498    0.692615
499    0.103792
500    0.908184
Name: EV/EBITDA Percentile, Length: 501, dtype: object
0      0.796407
1      0.037924
2      0.141717
3      0.904192
4      0.680639
         ...   
4

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.10,N/A,35.40,0.744511,7.87,0.786427,6.420,0.808383,24.063203,0.790419,12.167982,0.796407,N/A
1,AAL,17.22,N/A,-5.68,0.03992,-1.61,0.05988,0.391,0.01996,-11.891548,0.053892,1.210736,0.037924,N/A
2,AAP,226.56,N/A,23.29,0.479042,4.22,0.5998,1.200,0.157685,12.309128,0.403194,2.702114,0.141717,N/A
3,AAPL,172.69,N/A,28.66,0.618762,40.14,0.974052,7.410,0.846307,22.744446,0.774451,17.662805,0.904192,N/A
4,ABBV,174.73,N/A,27.78,0.602794,19.42,0.932136,5.440,0.748503,13.308175,0.45509,9.705364,0.680639,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,121.30,N/A,23.75,0.491018,-4.12,0.055888,5.240,0.729541,19.794906,0.700599,9.268269,0.662675,N/A
497,ZBH,131.45,N/A,70.90,0.942116,2.14,0.325349,3.540,0.531936,20.414940,0.714571,6.222539,0.401198,N/A
498,ZBRA,438.90,N/A,27.68,0.600798,7.77,0.783433,4.130,0.602794,19.635870,0.692615,8.797453,0.592814,N/A
499,ZION,64.64,N/A,6.42,0.095808,1.43,0.199601,2.820,0.438124,4.738746,0.103792,2.451010,0.121756,N/A


We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

In [ ]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)*100
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,139.10,N/A,35.40,0.744511,7.87,0.786427,6.420,0.808383,24.063203,0.790419,12.167982,0.796407,78.522954
1,AAL,17.22,N/A,-5.68,0.03992,-1.61,0.05988,0.391,0.01996,-11.891548,0.053892,1.210736,0.037924,4.231537
2,AAP,226.56,N/A,23.29,0.479042,4.22,0.5998,1.200,0.157685,12.309128,0.403194,2.702114,0.141717,35.628743
3,AAPL,172.69,N/A,28.66,0.618762,40.14,0.974052,7.410,0.846307,22.744446,0.774451,17.662805,0.904192,82.355289
4,ABBV,174.73,N/A,27.78,0.602794,19.42,0.932136,5.440,0.748503,13.308175,0.45509,9.705364,0.680639,68.383234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,121.30,N/A,23.75,0.491018,-4.12,0.055888,5.240,0.729541,19.794906,0.700599,9.268269,0.662675,52.794411
497,ZBH,131.45,N/A,70.90,0.942116,2.14,0.325349,3.540,0.531936,20.414940,0.714571,6.222539,0.401198,58.303393
498,ZBRA,438.90,N/A,27.68,0.600798,7.77,0.783433,4.130,0.602794,19.635870,0.692615,8.797453,0.592814,65.449102
499,ZION,64.64,N/A,6.42,0.095808,1.43,0.199601,2.820,0.438124,4.738746,0.103792,2.451010,0.121756,19.181637


Sort the dataframe according to the RV score



In [ ]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

In [ ]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,17.22,1161440,-5.68,0.03992,-1.6100,0.05988,0.3910,0.01996,-11.891548,0.053892,1.210736,0.037924,4.231537
1,UNM,32.01,624804,7.95,0.111776,0.5639,0.065868,0.5540,0.0499,3.954296,0.081836,0.541585,0.001996,6.227545
2,PRU,118.96,168123,6.04,0.085828,0.7350,0.071856,0.6343,0.063872,3.964687,0.083832,0.601506,0.007984,6.267465
3,GM,39.38,507872,5.87,0.077844,0.9931,0.091816,0.4587,0.035928,1.960874,0.05988,1.692474,0.081836,6.946108
4,PVH,73.21,273186,5.57,0.06986,0.9370,0.087824,0.5345,0.043912,5.019211,0.117764,1.119829,0.02994,6.986028
5,AIG,64.44,310366,5.80,0.073852,0.7640,0.073852,0.9801,0.113772,3.682513,0.073852,0.983757,0.01996,7.105788
6,C,51.20,390625,5.00,0.061876,0.5560,0.061876,1.2800,0.164671,3.231098,0.067864,1.184663,0.033932,7.804391
7,LNC,63.95,312744,8.54,0.123752,0.5634,0.063872,0.5792,0.057884,6.450062,0.159681,0.592602,0.005988,8.223553
8,F,15.01,1332445,3.44,0.053892,1.3000,0.177645,0.4580,0.033932,3.835341,0.07984,2.002328,0.0998,8.902196
9,MET,70.58,283366,9.81,0.149701,0.8630,0.077844,0.8473,0.095808,5.409362,0.129741,0.827167,0.017964,9.421158


In [ ]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)
writer.save()

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ["Company's Name", string_template],
                    'D': ['Number of Shares to Buy', integer_template],
                    'E': ['One-Year Price Return', percent_template],
                    'F': ['One-Year Return Percentile', percent_template],
                    'G': ['Six-Month Price Return', percent_template],
                    'H': ['Six-Month Return Percentile', percent_template],
                    'I': ['Three-Month Price Return', percent_template],
                    'J': ['Three-Month Return Percentile', percent_template],
                    'K': ['One-Month Price Return', percent_template],
                    'L': ['One-Month Return Percentile', percent_template],
                    'M': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)